## 接下来自顶向下，也就是从main方法开始，逐个分析各函数的算法流程和依赖关系。
## 1. train_pretrain.py

### 1.1 main()

#### 1.1.1 处理流程

- parser定义各种超参数

- args存储parser中的超参数

- 实例化MiniMindConfig为lm_config
- 设置存储路径等
- 设置token字典的长度
- 初始化ctx
- 配置分布式训练
- 初始化wandb
- 初始化model和tokenizer
- 初始化PretrainDataset: train_ds
- 初始化分布式样例
- 初始化DataLoader: train_loader
- 初始化scaler
- 初始化optimizer
- 调用train_epoch, 在for循环中开始训练

#### 1.1.2 依赖函数

| Name                          | Description                            |
| ----------------------------- | -------------------------------------- |
| **argparse**                  | 初始化一个parser，设置并存储各种超参数 |
| **MiniMindConfig**            | Minimind模型设置                       |
| **ctx**                       | 一个能够提高计算效率的计算器           |
| **wandb**                     | 一个数据看板和模型优化器               |
| **init_model**                | 初始化model和tokenizer                 |
| **PretrainDataset**           | 实例化预训练数据集                     |
| **DistributedSampler**        | 分布式训练相关                         |
| **DataLoader**                | 生成数据迭代器                         |
| **torch.cuda.amp.GradScaler** | 一个控制模型参数提高优化效率的计算器   |
| **optim**                     | 优化器                                 |
| **train_epoch**               | 单轮训练方法                           |

### 1.2 argparse

- 使用argparse.ArgumentParser实例化一个parser，

然后使用

- add_argument(name_or_flags = , type = , default = )

可以逐个往解析器中添加参数。

- parser.parse_args() 可以返回所有参数，以类似字典的形式。

**无显式的依赖函数**

### 1.3 MiniMindConfig

继承了PretrainConfig类，用于设置模型的各种参数。

**无显式的依赖函数**

### 1.4 ctx

一个更高效的计算器，使用它计算会极大提高效率

**无显式的依赖函数**

### 1.5 wandb

算是一个数据看板+模型调优的小工具

**无显式的依赖函数**

### 1.6 init_model

初始化tokenizer和model，并且打印Log

| Name                | Description        |
| ------------------- | ------------------ |
| AutoTokenizer       | 实例化tokenizer    |
| MiniMindForCausalLM | 实例化MiniMind模型 |

### 1.7 AutoTokenizer

读取json文件，实例化一个tokenizer

**无显式依赖函数**

### 1.8 MiniMindForCausalLM

**实例化一个主模型，具体结构再议**

### 1.9 PretrainDataset

继承了torch的Dataset类，读取指定路径的文件，实例化数据集。

**无显式的依赖**

### 1.10 DistributedSampler

略。

### 1.11 DataLoader

接收一个Dataset的子类实例和其他相关参数，生成数据迭代器（其实也可以用手动for循环代替）

**无显式的依赖**

### 1.12 torch.cuda.amp.GradScaler

动态管理梯度缩放的一个扫描器，在参数优化的时候把optimizer放在这里边跑，好处多多。

**无显式的依赖**

### 1.13 optim

优化器，老生常谈

**无显式的依赖**

### 1.14 train_epoch

接收一个轮次，一个看板实例wandb

- 实例化交叉熵计算器
- 记录开始计算的时间
- 载入数据迭代器，for每一个step
  - 将数据放到GPU上
  - 更新学习率
  - for optimizer里的每一个param
    - 更新学习率
  - 使用ctx计算
    - 模型当前的残差
    - 模型当前的loss
  - 使用scaler后向传播优化参数
  - 假如经过了特定步数的训练，要进行梯度更新控制
    - 取消缩放
    - 裁剪梯度
    - 参数更新
    - 更新缩放器
    - 清空梯度
  - 假如经过了特定步数的训练，记录训练日志
    - 修正损失值
    - 获取学习率
    - 估算计算时间
    - 同步日志到wandb
  - 对于专家模型有另外安排，此处略。

| Name                 | Description  |
| -------------------- | ------------ |
| **CrossEntropyLoss** | 交叉熵计算器 |
| **get_lr**           | 更新学习率   |
| **scaler**           | 梯度缩放器   |
| **optimizer**        | 优化器       |
| **Logger**           | 记录日志     |</br>
绘制一个不标准的UML图如下：

# 根据以上的拆解，我们开始尝试自底向上复现train_pretrain.py的各个模块。

In [ ]:
import os
import sys
__package__ = "trainer"
# sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '..')))

import time
import math
import warnings
import torch
from torch import optim, nn
from torch.utils.data import DataLoader
from contextlib import nullcontext
from transformers import AutoTokenizer
# from model.model_minimind import MiniMindConfig, MiniMindForCausalLM
# from dataset.lm_dataset import PretrainDataset

warnings.filterwarnings('ignore')

In [ ]:
def Logger(content):
  # 由于笔者暂时不考虑分布式训练，因此此处以及以下各处与源码有所不同。
  print(content)

In [ ]:
def get_lr(current_step, total_step, lr):
  # 余弦退火调度学习率，随着训练的进行，学习率逐渐减小但不至于为0
  return lr / 10 + 0.5 * lr * (1 + math.cos(math.pi * current_step / total_steps))

In [ ]:
def init_model(lm_config):
    # 自动读取目标目录的json文件，初始化为tokenizer对象
    tokenizer = AutoTokenizer.from_pretrained('../model/')
    model = MiniMindForCausalLM(lm_config).to(args.device)
    Logger(f'LLM可训练总参数量：{sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6:.3f} 百万')
    return model, tokenizer

In [ ]:
class Arguments:
  def __init__(self,
      out_dir = "../out",
      epochs = '1',
      batch_size = '32',
      learning_rate = '5e-4',
      device = "cuda" if torch.cuda.is_available() else "cpu",
      dtype = "bfloat16",
      accumulation_steps = 8,
      grad_clip = 1,
      warmup_iters = 0,
      log_interval = 100,
      save_interval = 100,
      local_rank = -1,
      hidden_size = 512,
      num_hidden_layers = 8,
      max_seq_len = 512,
      data_path = "../dataset/pretrain_hq.jsonl"
  ):
    self.out_dir = out_dir
    self.epochs = epochs
    self.batch_size = batch_size
    self.learning_rate = learning_rate
    self.device = device
    self.dtype = dtype
    self.accumulation_steps = accumulation_steps
    self.grad_clip = grad_clip
    self.warmup_iters = warmup_iters
    self.log_interval = log_interval
    self.save_interval = save_interval
    self.local_rank = local_rank
    self.hidden_size = hidden_size
    self.num_hidden_layers = num_hidden_layers
    self.max_seq_len = max_seq_len
    self.data_path = data_path
    self.save_dir = None
    self.tokens_per_iter = self.batch_size * self.max_seq_len

In [ ]:
def output_logger():
    if step % args.log_interval == 0:
        spend_time = time.time() - start_time
        Logger(
            'Epoch:[{}/{}]({}/{}) loss:{:.3f} lr:{:.12f} epoch_Time:{}min:'.format(
                epoch + 1,
                args.epochs,
                step,
                iter_per_epoch,
                loss.item() * args.accumulation_steps,
                optimizer.param_groups[-1]['lr'],
                spend_time / (step + 1) * iter_per_epoch // 60 - spend_time // 60))

In [ ]:
def train_epoch_new(epoch, train_loader):
  # 1. 更新学习率
  # 2. 计算loss
  # 3. 更新参数
  # 初始化一个loss_function
  loss_fct = nn.CrossEntropyLoss(reduction='none')
  start_time = time.time()

  # 取出X, Y, loss_mask
  for idx, (X, Y, loss_mask) in enumerate(train_loader):
    X = X.to(args.device)
    Y = Y.to(args.device)
    loss_mask = loss_mask.to(args.device)

    # 更新学习率
    lr = get_lr(epoch * iter_per_epoch + step, args.epochs * iter_per_epoch, args.learning_rate)
    for param_group in optim.param_groups:
      param_group['lr'] = lr

    # 把计算过程用ctx套起来
    with ctx:
      # 计算掩码后loss
      result = model(X)
      loss = loss_fct(result.view(-1, result.logits.size(-1)), Y.view(-1)).view(Y.size())
      loss = (loss * loss_mask).sum() / loss_mask.sum()
      loss += result.aux_Loss

      # 优化参数
      optim.zero_grad()
      scaler.scale(loss).backward()
      scaler.step(optim)
      scaler.update()

      # 打印loss等
      output_logger()

## 在首先复现了最简单的Logger、get_lr和init_model这三个函数之后，在main函数时，笔者卡住了。
倒不是看不懂代码，而是这套流程里调用了大量的包，一时不好复现（这要求你对这些包都有一定的了解）。
接着考虑，对代码进行删减，由于我们这里暂时不讨论分布式训练和对专家模型的复现，因此可以把这部分删除，然后开始学习main当中其他的包，应该如何使用。

In [ ]:
# 拆解后的代码如下：
if __name__ == "__main__":
    args = Arguments()
    lm_config = MiniMindConfig(hidden_size=args.hidden_size, num_hidden_layers=args.num_hidden_layers)
    # args.save_dir = os.path.join(args.out_dir)
    # os.makedirs(args.save_dir, exist_ok=True)
    # os.makedirs(args.out_dir, exist_ok=True)

    # token字典 长度设置
    device_type = args.device

    ctx = nullcontext() if device_type == "cpu" else torch.cuda.amp.autocast()

    tokens_per_iter = args.tokens_per_iter

    # 初始化模型和tokenizer
    model, tokenizer = init_model(lm_config)
    # 初始化Dataset和DataLoader
    train_ds = PretrainDataset(args.data_path, tokenizer, max_length=args.max_seq_len)
    train_loader = DataLoader(
        train_ds,
        batch_size=args.batch_size,
        pin_memory=True,
        drop_last=False,
        shuffle=False,
        num_workers=args.num_workers
    )
    # 初始化scaler和optimizer
    scaler = torch.cuda.amp.GradScaler()
    optimizer = optim.AdamW(model.parameters(), lr=args.learning_rate)
    iter_per_epoch = len(train_loader)
    for epoch in range(args.epochs):
        train_epoch_new(epoch, train_loader)
'''
在main中出现的非自定义函数还有：
torch.cuda.amp.autocast
PretrainDataset
DataLoader
torch.cuda.amp.GradScaler
optim
接下来我们逐个学习、熟悉一下它们。
'''

'\n在main中出现的非自定义函数还有：\ntorch.cuda.amp.autocast\nPretrainDataset\nDataLoader\ntorch.cuda.amp.GradScaler\noptim\n接下来我们逐个学习、熟悉一下它们。\n'